In [20]:
%pip install torch transformers
%pip install ipywidgets
%pip install -U "huggingface_hub[cli]"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AdamW
from transformers import AutoModelForCausalLM
from huggingface_hub import notebook_login
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import re

In [22]:
MODEL_NAME = "EleutherAI/gpt-neo-125M"  
LEARNING_RATE = 1e-4
EPOCHS = 20
BATCH_SIZE = 6
MAX_LEN = 50
FILE_NAME = 'model'

In [23]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token="")
tokenizer.pad_token = tokenizer.eos_token  

In [24]:
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=MAX_LEN):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encodings = self.tokenizer(
            self.texts[idx], truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt"
        )
        input_ids = encodings["input_ids"].squeeze()
        attention_mask = encodings["attention_mask"].squeeze()
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": input_ids}

In [25]:
data = open('data/goida.txt').read() # + open('data/doc77.txt').read()
texts = re.split(r'[.!?]', data)
texts = [t.strip() for t in texts if t]

In [26]:
dataset = TextDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [27]:
class SimpleTransformerModel(nn.Module):
    def __init__(self, model_name):
        super(SimpleTransformerModel, self).__init__()
        self.transformer = AutoModelForCausalLM.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss if labels is not None else None
        logits = outputs.logits
        return loss, logits

In [28]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

/Users/nickolasbykovskij/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [30]:
def train_model(model, dataloader, optimizer, epochs=EPOCHS):
    model.train()
    total_steps = len(dataloader) * epochs 
    progress_bar = tqdm(total=total_steps, desc="Training Progress")

    for epoch in range(1, epochs + 1):
        progress_bar.set_description(f"Training Progress ")

        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss  

            loss.backward()
            optimizer.step()

            progress_bar.update(1)
            progress_bar.set_postfix(loss=loss.item(), epoch=epoch)

    progress_bar.close()

In [31]:
# train_model(model, dataloader, optimizer)

In [32]:
def generate_text(model, tokenizer, prompt, max_len=MAX_LEN, temperature=0.7, top_k=50, top_p=0.9):
    model.eval()
    input_ids = tokenizer(prompt, return_tensors="pt", padding=True).input_ids

    attention_mask = input_ids != tokenizer.pad_token_id

    outputs = model.generate(
        input_ids,
        # attention_mask=attention_mask,
        max_length=max_len,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        num_return_sequences=1,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,  
        use_cache=False
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


In [ ]:
torch.cuda.empty_cache()

prompt = ''
text = generate_text(model, tokenizer, prompt, temperature=0.7, max_len=100).replace('\n', "")

text = re.split(r'[.!?]"', text)[0]

print(text)

Go fuck yourself, you're gonna get the fuck out of your ass.I'm not gonna lie, I don't know what I'm going to do. I don't know what I'm going to do.I'm not going to lie. I'm not going to lie.I'm not going to lie. I'm not going to lie.I'm not going to lie. I'm not going to lie.I'm not going to


In [34]:
torch.save(model.state_dict(), f'models/{FILE_NAME}.pth')